## Homework #4

In [1]:
!pip install -r requirements.txt

In [4]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

import requests

##### Параметры

In [ ]:
RANDOM_STATE=1234
DATASET_URL=

##### Вспомогательные функции

In [5]:
def download_file(url, filename):
    r = requests.get(url, allow_redirects=True)
    open(filename, 'wb').write(r.content)

#### Реализовать классификацию двумя моделями: CNN, LSTM


#### Сравнить качество обученных моделей